#### *ML-7.4: Оптимизация гиперпараметров. Практика*

Наша практика будет основана на соревновании Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа).

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

## **1. Оценка данных**

In [1]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных

from sklearn import linear_model #линейные модели
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV # оптимизация
from sklearn.model_selection import RandomizedSearchCV # оптимизация
from sklearn.model_selection import cross_val_score
import hyperopt # оптимизация
from hyperopt import hp, fmin, tpe, Trials
import optuna # оптимизация

In [2]:
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Смотрим на сбалансированность классов:

In [3]:
data['Activity'].value_counts(normalize=True)

1    0.542255
0    0.457745
Name: Activity, dtype: float64

Есть небольшой дисбаланс, поэтому учтем это при разбиении на тренировочную и тестовую выборки.

Создаем матрицу наблюдений $X$ и вектор ответов $y$

In [4]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

Разделяем выборку на тренировочную и тестовую в соотношении 80/20. Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение).

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

## **2. Оптимизация гиперпараметров модели**

## **2.1 Логистическая регрессия**

### <center> **Базовая модель**

Зафиксируем только те метрики, которые были получены без дополнительной настройки, то есть со значениями гиперпараметров, установленных по умолчанию:

In [6]:
# Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=50)  

# Обучаем модель
log_reg.fit(X_train, y_train)

# Оценка качества модели
y_train_pred = log_reg.predict(X_train)  
y_test_pred = log_reg.predict(X_test)

print('f1_score на тренировочной наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочной наборе: 0.87
f1_score на тестовом наборе: 0.79


c:\Users\An\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### <center> **Оптимизация гиперпараметров GridSearchCV**

In [45]:
# Сетка гиперпараметров
param_grid_1 = [
              {'penalty': ['l2'] , 
               'solver': ['lbfgs', 'sag', 'newton-cg'], 
               'C': [0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.3, 0.5, 0.7, 0.9, 1]},  
              
              {'penalty': ['l1'] , 
               'solver': ['liblinear', 'saga'], 
               'C': [0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}                
]

# Поиск по сетке параметров с кросс-валидацией
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50),                                          
    param_grid=param_grid_1,
    scoring = 'f1',
    cv=5, 
    n_jobs = -1
)  

# Обучаем модель 
grid_search_1.fit(X_train, y_train) 

# Оценка качества модели
y_train_pred = grid_search_1.predict(X_train)
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))
print("f1_score при кросс-валидации: {:.2f}".format(grid_search_1.best_score_))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("f1_score на тестовом наборе: {:.2f}".format(grid_search_1.score(X_test, y_test)))


Наилучшие значения гиперпараметров: {'C': 0.03, 'penalty': 'l2', 'solver': 'sag'}
f1_score при кросс-валидации: 0.78
f1_score на тренировочном наборе: 0.83
f1_score на тестовом наборе: 0.80


c:\Users\An\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Значение метрики f1_score на тестовом наборе улучшилось

### <center> **Оптимизация гиперпараметров RandomizedSearchCV**

In [43]:
# Пространство параметров: создаем несколько словарей с наборами параметров
param_distributions_1 = [
                    {'penalty': ['l2'] ,      
                    'solver': ['lbfgs', 'sag', 'newton-cg'],    
                    'C': [0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.3, 0.5, 0.7, 0.9, 1]},   
              
                    {'penalty': ['l1'] ,
                    'solver': ['liblinear', 'saga'],
                    'C': [0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.3, 0.5, 0.7, 0.9, 1]},                                       
]

# Случайный поиск по распределению параметров с кросс-валидацией            
random_search_1 = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions_1,
    scoring = 'f1', 
    cv=5, 
    n_iter = 20, 
    n_jobs = -1
)   

# Обучаем модель 
random_search_1.fit(X_train, y_train)    

# Оценка качества модели
y_train_pred = random_search_1.predict(X_train)

print("Наилучшие значения гиперпараметров: {}".format(random_search_1.best_params_))
print("f1_score при кросс-валидации: {:.2f}".format(random_search_1.best_score_))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("f1_score на тестовом наборе: {:.2f}".format(random_search_1.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.03}
f1_score при кросс-валидации: 0.78
f1_score на тренировочном наборе: 0.83
f1_score на тестовом наборе: 0.80


c:\Users\An\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Значение метрики f1_score на тестовом наборе улучшилось

### <center> **Оптимизация гиперпараметров HYPEROPT**

In [13]:
# зададим пространство поиска гиперпараметров
space_1 = {'penalty': hp.choice('penalty', ['l2']),  
            'solver': hp.choice('solver', ['lbfgs', 'sag', 'newton-cg']),  
            'C': hp.quniform('C', 0.01, 0.1, 0.01)  
            }

# Зафксируем random_state
random_state = 42

# Целевая функция
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {'penalty': (params['penalty']), 
              'solver': (params['solver']), 
             'C': float(params['C'])
              }
  
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=50)
      
    # Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean() 
       
    # Метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# Начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr,  
          space=space_1, 
          algo=tpe.suggest, 
          max_evals=20,      
          trials=trials, 
          rstate=np.random.default_rng(random_state)
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [02:13<00:00,  6.69s/trial, best loss: -0.7843404423441448]
Наилучшие значения гиперпараметров {'C': 0.03, 'penalty': 0, 'solver': 1}


In [15]:
# рассчитаем точность для тестовой выборки
hyperopt_lr = linear_model.LogisticRegression(
    random_state=random_state,
    penalty='l2',   
    solver='sag',  
    C=0.03,
    max_iter=50
)
 
hyperopt_lr.fit(X_train, y_train) 

# Оценка качества модели
best_score = (np.array(list(x['result']['loss'] 
                            for x in trials.trials)) * (-1)).max()
print('f1_score на кросс-валидации: {:.2f}'.format(best_score))

y_train_pred = hyperopt_lr.predict(X_train)
y_test_pred = hyperopt_lr.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на кросс-валидации: 0.78
f1_score на тренировочном наборе: 0.83
f1_score на тестовом наборе: 0.80


c:\Users\An\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Значение метрики f1_score на тестовом наборе улучшилось

### <center> **Оптимизация гиперпараметров OPTUNA**

In [23]:
# Целевая функция
def optuna_lr(trial, cv=5):
    # пространство гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l2'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sag', 'newton-cg'])
    C = trial.suggest_float('C', 0.01, 0.1, step=0.01)
    
    model = linear_model.LogisticRegression(random_state=random_state, 
                                            penalty=penalty, 
                                            solver=solver, 
                                            C=C, 
                                            max_iter=50)
    
    # Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    
    return score 

# Создаем объект исследования, максимизируем метрику
study_lr = optuna.create_study(study_name="LogisticRegression", direction="maximize")

# Не выводить информационный журнал
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Поиск оптимальных гиперпараметров
study_lr.optimize(optuna_lr, n_trials=20)  

In [24]:
# рассчитаем точность для тестовой выборки
optuna_lr = linear_model.LogisticRegression(**study_lr.best_params,random_state=random_state, max_iter=50)
optuna_lr.fit(X_train, y_train)

# Оценка качества модели
y_train_pred = optuna_lr.predict(X_train)
y_test_pred = optuna_lr.predict(X_test)

print("Наилучшие значения гиперпараметров {}".format(study_lr.best_params))
print('f1_score на кросс-валидации: {:.2f}'.format(study_lr.best_value))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'sag', 'C': 0.03}
f1_score на кросс-валидации: 0.79
f1_score на тренировочном наборе: 0.83
f1_score на тестовом наборе: 0.80


c:\Users\An\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Значение метрики f1_score на тестовом наборе улучшилось

### Вывод:
Для модели логистической регрессии удалось улучшить метриу f1_score всеми четырьмя методами.

## 2.2 **Случайный лес**

### <center> **Базовая модель**

Зафиксируем только те метрики, которые были получены без дополнительной настройки, то есть со значениями гиперпараметров, установленных по умолчанию:

In [5]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)

# Оценка качества модели 
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.81


### <center> **Оптимизация гиперпараметров GridSearchCV**

In [6]:
# Сетка параметров
param_grid_2 = [
              {'n_estimators': list(range(100, 301, 50)), 
               'max_depth': list(range(10, 51, 5)), 
               'min_samples_leaf': list(range(1, 10, 2)), 
               'criterion': ['gini', 'entropy']}                
]

# Поиск по сетке параметров с кросс-валидацией 
grid_search_2 = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid_2,
    scoring = 'f1', 
    cv=5, 
    n_jobs = -1
)  
 
# Обучаем модель 
grid_search_2.fit(X_train, y_train) 

# Оценка качества модели
y_train_pred = grid_search_2.predict(X_train)
print("Наилучшие значения гиперпараметров: {}".format(grid_search_2.best_params_))
print("f1_score при кросс-валидации: {:.2f}".format(grid_search_2.best_score_))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("f1_score на тестовом наборе: {:.2f}".format(grid_search_2.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'n_estimators': 300}
f1_score при кросс-валидации: 0.82
f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.83


Значение метрики f1_score на тестовом наборе улучшилось

### <center> **Оптимизация гиперпараметров RandomizedSearchCV**

In [7]:
# Пространство параметров
param_distributions_2 = [
              {'n_estimators': list(range(100, 401, 1)), 
               'max_depth': list(range(10, 51, 1)), 
               'min_samples_leaf': list(range(1, 10, 1)), 
               'criterion': ['gini', 'entropy']} 
]

# Случайный поиск по распределению параметров с кросс-валидацией            
random_search_2 = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions_2,
    scoring = 'f1', 
    cv=5, 
    n_iter = 20, 
    n_jobs = -1
)   

# Обучаем модель 
random_search_2.fit(X_train, y_train) 

# Оценка качества модели
y_train_pred = random_search_2.predict(X_train)

print("Наилучшие значения гиперпараметров: {}".format(random_search_2.best_params_))
print("f1_score при кросс-валидации: {:.2f}".format(random_search_2.best_score_))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("f1_score на тестовом наборе: {:.2f}".format(random_search_2.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'n_estimators': 216, 'min_samples_leaf': 2, 'max_depth': 16, 'criterion': 'entropy'}
f1_score при кросс-валидации: 0.82
f1_score на тренировочном наборе: 0.99
f1_score на тестовом наборе: 0.83


Значение метрики f1_score на тестовом наборе улучшилось

### <center> **Оптимизация гиперпараметров HYPEROPT**

In [10]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 401, 1),
       'max_depth' : hp.quniform('max_depth', 10, 51, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1),
       'criterion': hp.choice('criterion', ['gini', 'entropy'])
      }

# зафксируем random_state
random_state = 42

# Целевая функция
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):  
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
              'min_samples_leaf': int(params['min_samples_leaf']),
              'criterion': params['criterion']
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    
    # Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()  

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [11]:
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf,  
          space=space, 
          algo=tpe.suggest, 
          max_evals=20, 
          trials=trials, 
          rstate=np.random.default_rng(random_state) 
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [03:01<00:00,  9.05s/trial, best loss: -0.8138225777493373]
Наилучшие значения гиперпараметров {'criterion': 1, 'max_depth': 28.0, 'min_samples_leaf': 2.0, 'n_estimators': 327.0}


In [15]:
# Расчет метрики для лучших найденных гиперпараметров  
hyperopt_rf = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),   
    max_depth=int(best['max_depth']),
    criterion='entropy',
    min_samples_leaf=int(best['min_samples_leaf'])
)
 
hyperopt_rf.fit(X_train, y_train) 

# Оценка качества модели
best_score = (np.array(list(x['result']['loss'] 
                            for x in trials.trials)) * (-1)).max()
print('f1_score на кросс-валидации: {:.2f}'.format(best_score))

y_train_pred = hyperopt_rf.predict(X_train)
y_test_pred = hyperopt_rf.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на кросс-валидации: 0.81
f1_score на тренировочном наборе: 0.99
f1_score на тестовом наборе: 0.83


Значение метрики f1_score на тестовом наборе улучшилось

### <center> **Оптимизация гиперпараметров OPTUNA**

In [25]:
# Целевая функция
def optuna_rf(trial, cv=5):
    # Пространство гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 100, 401, 1)
    max_depth = trial.suggest_int('max_depth', 10, 51, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, 1)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators, 
                                            max_depth=max_depth, 
                                            min_samples_leaf=min_samples_leaf, 
                                            criterion=criterion, 
                                            random_state=random_state)
    
    # Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    
    return score 

# Создаем объект исследования, максимизируем метрику
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")

# Не выводить информационный журнал
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Поиск оптимальных гиперпараметров
study_rf.optimize(optuna_rf, n_trials=20)

In [26]:
# рассчитаем точность для тестовой выборки
optuna_rf = ensemble.RandomForestClassifier(**study_rf.best_params,random_state=random_state)
optuna_rf.fit(X_train, y_train)

# Оценка качества модели
y_train_pred = optuna_rf.predict(X_train)
y_test_pred = optuna_rf.predict(X_test)

print("Наилучшие значения гиперпараметров {}".format(study_rf.best_params))
print('f1_score на кросс-валидации: {:.2f}'.format(study_rf.best_value))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'n_estimators': 266, 'max_depth': 15, 'min_samples_leaf': 3, 'criterion': 'gini'}
f1_score на кросс-валидации: 0.82
f1_score на тренировочном наборе: 0.97
f1_score на тестовом наборе: 0.82


Значение метрики f1_score на тестовом наборе улучшилось

### Вывод:
Для модели случайного леса удалось улучшить метриу f1_score всеми четырьмя методами.